In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [2]:
# Load GT detections
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

# Load computed detections
detection_path = 'retinanet_101_detections_trained.pkl'
data_path = '../../AICity_data/train/S03/c010/'

with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

# Load video frames
video_path = '../../AICity_data/train/S03/c010/vdo.avi' #UPDATE PATH WHEN PUSHING
vidcap = cv2.VideoCapture(video_path)
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

## Overlapping boxes approach

In [ ]:
detections_pd = get_detection_dataframe(all_detections['0'], firstFrame=True)
detections_pd

In [ ]:
detections_pd = get_detection_dataframe(all_detections['1'])
detections_pd

In [3]:
def get_detection_dataframe(detections, iclLineAndUpdate = True, firstFrame = False):
    bboxes = []
    bsizes = []
    lines = deque(maxlen=32)
    bdetections = []

    if firstFrame == True:
        tracks = list(range(1, len(detections)+1))
    else:    
        tracks = [0]*len(detections)

    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
        
    updated = [False]*len(detections)
    use_of = [0]*len(detections)
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)
        
        centers = []
        centers.append(get_box_center(bbox))
        lines.append(centers)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    if iclLineAndUpdate == True: 
        detec = {
            'track': tracks,
            'detection': bdetections,
            'bbox': bboxes,
            'size': bsizes,
            'line': lines,
            'colour': colours,
            'updated': updated,
            'opt_flow': use_of
        }
    else:
        detec = {
            'track': tracks,
            'detection': bdetections,
            'bbox': bboxes,
            'size': bsizes,
            'colour': colours,
        }
    detections_pd = pd.DataFrame(detec)
    detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [4]:
def update_track3(detections_pd, previous_detections_pd, tolerance = 0.5, imgs = None):
    detections_pd['updated'] = False
    detections_pd = previous_detections_pd.reset_index(drop=True)

    previous_tracks = []

    for index, previous_detection in previous_detections_pd.iterrows():
        length, _ = detections_pd.shape

        IoUlist = []
        for i in range(length):
            IoU = previous_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)

        indexMax = IoUlist.index(max(IoUlist))

        if max(IoUlist) > tolerance and detections_pd.at[indexMax, 'updated'] != True:
            detections_pd.at[indexMax, 'track'] = previous_detection['track']
            # add line
            detections_pd.at[indexMax, 'updated'] = True

        previous_tracks.append(previous_detection['track'])

    idxs = detections_pd.index[detections_pd['updated'] == False].tolist() #Non-updated detections
    track_check = 1
    for ind in idxs:
        while track_check in previous_tracks:
            track_check += 1
        detections_pd.at[ind, 'track'] = track_check
        previous_tracks.append(track_check)
            
    return detections_pd

In [5]:
detection_history = []

# Get the first frame
detections_prev_pd = get_detection_dataframe(all_detections['0'], firstFrame = True)

# Pre-process the first frame if needed
detections_prev_pd = remove_overlaps(detections_prev_pd, 0.9)
detection_history.append(detections_prev_pd)

_, frame_0 = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    
    detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = remove_overlaps(detections_pd, 0.9)

    detections_pd = update_track3(detections_pd, detections_prev_pd, tolerance=0.5)
    detection_history.append(detections_pd)

    detections_prev_pd = detections_pd

pkl.dump(detection_history, open('tracking_history3.pkl', 'wb'))

100%|██████████| 2139/2139 [02:47<00:00, 12.81it/s]


In [18]:
def update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    
    # Loop each new detection
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps with max IoU and update if found
        IoUlist = []
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)
            
        indexMax = IoUlist.index(max(IoUlist))
            
        if max(IoUlist) > tolerance and detections_pd.at[indexMax,'updated'] != True:
            detections_pd.at[indexMax,'detection'] = next_detection['detection']
            detections_pd.at[indexMax,'bbox'] = next_detection['bbox']
            detections_pd.at[indexMax,'size'] = next_detection['size']
            detections_pd.at[indexMax,'line'].append(next_detection['line'][0])
            detections_pd.at[indexMax,'updated'] = True
            detections_pd.at[indexMax, 'opt_flow'] = 0
            next_detections_pd.at[index, 'updated'] = True
    


    # If frame images "imgs" are provided, use optical flow approach to improve tracking
    if imgs is not None:
        idxs = detections_pd.index[detections_pd['updated'] == False].tolist() #Non-updated detections
        for ind in idxs:
            if detections_pd.at[ind, 'opt_flow'] < 1: # After _ consecutive frames using OF, remove detection
                box = detections_pd.at[ind, 'bbox']
                block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]

                #Compute optical flow of the not updated tracking bounding box
                of = compute_block_of(block, box, imgs[1], 'backward', 10)
                
                #Update bounding box
                if len(of) > 0:
                    if (of[0] != 0.0 or of[1] != 0.0):
                        newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
                        detections_pd.at[ind, 'bbox'] = newBox
                        detections_pd.at[ind, 'updated'] = True # NEED TO DOUBLE CHECK THIS
                        detections_pd.at[ind, 'detection'].updateBBox(newBox)
                        detections_pd.at[ind, 'line'].append(get_box_center(newBox))
                        detections_pd.at[ind, 'opt_flow'] = detections_pd.at[ind, 'opt_flow'] + 1


    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    counter = 0
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated'] == False]
        new_pd = new_pd.reset_index(drop=True)
        
        # Generate new track number      
        for i in range(len(new_pd)):
            while counter in detections_pd['track'].tolist():
                counter = counter + 1
            new_pd.at[i, 'track'] = counter
            counter = counter + 1

        # Add new tracks
        detections_pd = pd.concat([detections_pd, new_pd])
            
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [19]:
ims = []
detection_history = []
gif = False

# if gif:
#     fig, ax = plt.subplots()
#     plt.axis('off')

# Get the first frame
detections_pd = get_detection_dataframe(all_detections['0'])

# Pre-process the first frame if needed
detections_pd = remove_overlaps(detections_pd, 0.9)
detection_history.append(detections_pd)

_, past_frame = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    next_detections_pd = remove_overlaps(next_detections_pd, 0.9)
    
    detections_pd = update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=[past_frame, image])
    detection_history.append(detections_pd)
    past_frame = image

    # if gif:
    #     for index, row in detections_pd.iterrows():
    #         image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
    #     im = ax.imshow(image, animated=True)
    #     ims.append([im])

# if gif:
#     ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
#     ani.save('all_detects' + ".gif", writer=animation.PillowWriter(fps=24))
    
pkl.dump(detection_history, open('tracking_history.pkl', 'wb'))

100%|██████████| 2139/2139 [03:50<00:00,  9.29it/s]


## Kalman filter approach

In [5]:
# # Create Kalman tracker
# mot_tracker = Sort(max_age=10, min_hits=3, iou_threshold=0.3) # Sort Kalman tracker with default values

# results = {}

# # Iterate through frames
# for frame in tqdm(range(num_frames), desc = "Tracking objects each frame..."):
#     dets = []

#     _, im = vidcap.read() # Read frame
#     detections = get_detection_dataframe(all_detections[str(frame)], iclLineAndUpdate = False).sort_values("track") # Load detections

#     # Iterate through frame's detections
#     for (track_id, det, bbox, size, colour) in detections.itertuples(index=False):
#         dets.append(np.array(bbox))

#     trackers = mot_tracker.update(np.array(dets)) # Update tracker with current detections

#     for d in trackers: # Store new bboxes
#         d = d.astype(np.int32)
#         if frame not in results:
#             results[frame] = {d[4] % 47: {"bbox": [d[0], d[1], d[2], d[3]]}}
#         else:
#             results[frame][d[4] % 47] = {"bbox": [d[0], d[1], d[2], d[3]]}

# # Save Results to Disk
# pkl.dump(results, open("sort_bbox.pkl", "wb"))

Tracking objects each frame...: 100%|██████████| 2141/2141 [00:22<00:00, 95.27it/s] 


In [6]:
# # Pkl to pd.DataFrame
# colours = np.random.rand(47, 3) #Generate Random Colors
# old_pkl = pkl.load(open("sort_bbox.pkl", "rb"))
# new_pkl = []
# df = None

# for frame in old_pkl.keys():
#     new_data = {"frame": [],"track": [], "bbox": [], "colour": [], "size": [], "detection": []}
#     old_data = old_pkl[frame]
    
#     for track_id in old_data.keys():
#         new_data["track"].append(track_id)
#         bbox = old_data[track_id]["bbox"]
#         new_data["bbox"].append(bbox)
#         new_data["frame"].append("Frame {}".format(frame))
#         new_data["colour"].append(np.round(colours[track_id]*255).astype(np.uint8))
#         new_data["size"].append(int(np.abs(bbox[0] - bbox[2]) * np.abs(bbox[1] - bbox[3])))
#         new_data["detection"].append(VehicleDetection(frame=frame, ID=track_id, width=np.abs(bbox[0] - bbox[2]), conf=0.5,
#                                                       height=np.abs(bbox[1] - bbox[3]), left=bbox[0], right=bbox[2], top=bbox[1], bot=bbox[3]))
    
#     df = pd.DataFrame.from_dict(new_data).sort_values(by="track")

#     new_pkl.append(df)

# pkl.dump(new_pkl, open("kalman_tracking.pkl", "wb"))

## Evaluation

In [6]:
total_frames = 2141
initial_frame = 535

In [7]:
detections = {}
with open('tracking_history3.pkl','rb') as openFile:
    detections = pkl.load(openFile)

In [9]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    acc, frame_id = tracking_acc(frame, gt_detect, detection_history, acc)

display_metrics(acc)

100%|██████████| 1605/1605 [01:10<00:00, 22.87it/s]


     num_frames  precision    recall      idp       idr      idf1
acc        1605    0.77134  0.999317  0.77134  0.999317  0.870652


In [12]:
# detectionsK = {}
# with open('kalman_tracking.pkl','rb') as openFile:
#     detectionsK = pkl.load(openFile)

In [13]:
# total_frames = 2141
# initial_frame = 535

# accK = create_accumulator()

# for frame in tqdm(range(initial_frame, total_frames-1)):
#     accK, frame_id = tracking_acc(frame, gt_detect, detectionsK, accK)

# display_metrics(accK)

100%|██████████| 1605/1605 [01:07<00:00, 23.87it/s]


     num_frames  precision    recall       idp       idr      idf1
acc        1605   0.729659  0.999503  0.634604  0.869295  0.733637
